In [1]:
%run constants.ipynb
# The assumptions framework in sympy does not work so well with linear inequalities
# We use our own LP framework to detect constraints that violate V
from sympy import symbols, And, Not, simplify

In [2]:
EPS = 1e-65
def nonstricten(*ineqs):
    nonstrict = []
    for i in ineqs:
        if isinstance(i, Gt):
            # a>b => a >= b+EPS
            nonstrict.append(Ge(i.lhs, i.rhs + EPS))
        elif isinstance(i, Lt):
            # a<b => a <= b-EPS
            nonstrict.append(Le(i.lhs, i.rhs - EPS))
        else:
            nonstrict.append(i)  # already non-strict
    return nonstrict

In [3]:
from sympy.solvers.simplex import lpmax, InfeasibleLPError
from sympy.core import *

def solve_lp(*constraints, Ax=0):
    ''' Returns max LP solution for Ax <= b, under the given `constraints`. '''
    return lpmax(Ax, nonstricten(*constraints))  # flatten

def feasible(*ineqs, r=V):
    ''' Returns true if the linear inequalities `ineqs` are feasible under constraints `r`, 
        false otherwise. 
    '''
    try:
        result = solve_lp(*r,*ineqs, Ax=0)
    except InfeasibleLPError:
        return False
    return result

def redundant(ineq, r=V):
    ''' Returns true if the linear inequality `ineq` is redundant under constraints `r`, 
        false otherwise. 
    '''
    # if (Assumptions => Ineq) = (not A or I) is valid, then (A and not I) is unsatisfiable
    return feasible(Not(ineq), r=r) == False

def non_redundant(*ineqs, r=V):
    ''' Returns the non-redundant inequalities from `ineqs` under constraints `r`.
    '''
    return [i for i in ineqs if not redundant(i, r=r)]

def equiv(e1, e2):
    ''' Returns true if e1 = e2, false otherwise.
    '''
    return simplify(e1).equals(simplify(e2))

In [5]:
def reduce(inds, r=V):
    ''' 
    Reduces the 2D list of `inds` under restriction `r`.
    i.e. Inferrable indicators from `r` are converted to 0s or 1s.
    Returns a 2-tuple of constant value (evaluated indicators), remaining indicators.
    '''
    reduced = []
    const = 0
    for i in inds:
        if not feasible(*i, r=r):
            # not feasible
            continue
        else:
            # unsure
            rem = non_redundant(*i, r=r)
            if not rem:
                const += 1
            reduced.append(rem)
                
    return const, reduced